In [2]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [3]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [4]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [5]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [62]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [63]:
data.shape

(988, 5)

In [8]:
data.head(3)

,content,keywords,summary,title,url
0,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","[школа, образовательные стандарты, литература, история, фгос]","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html
1,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...","[красота, законы]",О живительной пользе укорота при выборе между плохим и хорошим,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html
2,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...","[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [64]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [65]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [66]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [67]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.05
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [68]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.03
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [69]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [70]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                      [ольга, васильева, обещала, "нг", не, перегружать, школьников]
1                                           [у, красоты, собственные, закон, и, воля]
2                                                            [апокалиптический, бунт]
3                   [f1., предсказать, результаты, гран-при, испании, было, несложно]
4                                               [возвращение, в, небесное, отечество]
5                                       [практическая, медицина, с, большим, будущим]
6                     [бумажный, носитель., вселенная,, человек,, язык, человеческий]
7                                      [в, багдаде, неспокойно,, а, будет, еще, хуже]
8    [ученые, предсказывают, появление, интуиции, у, искусственных, нейронных, сетей]
9                                  [россияне, от, вступления, в, вто, не, пострадают]
Name: title, dtype: object

In [71]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                        [и, в, –, по, на, что, с, о, ольга, к]
1                [и, не, в, что, –, на, красота, вы, если, как]
2                    [в, и, не, на, от, –, пепеляев, но, из, о]
3                       [в, не, и, на, что, но, это, с, как, к]
4                         [и, в, –, не, он, с, на, что, а, его]
5                [и, в, на, –, что, с, не, это, медицины,, уже]
6                          [–, и, в, не, на, из, что, по, с, а]
7                         [в, и, не, на, но, ли, что, а, до, -]
8                          [и, в, на, о, с, –, что, а, как, ai]
9    [в, на, и, по, что, кристалина, –, россии, георгиева, для]
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [72]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [73]:
data['content_norm'] = data['content'].apply(normalize)

In [74]:
data['title_norm'] = data['title'].apply(normalize)

In [75]:
data['title_norm'].head(10)

0                           [ольга, васильев, обещать, нг, перегружать, школьник]
1                                             [красота, собственный, закон, воля]
2                                                        [апокалиптический, бунт]
3                       [f1, предсказать, результат, гран-при, испания, несложно]
4                                              [возвращение, небесный, отечество]
5                                      [практический, медицина, больший, будущий]
6                    [бумажный, носитель, вселенная, человек, язык, человеческий]
7                                                     [багдад, неспокойно, худой]
8    [учёный, предсказывать, появление, интуиция, искусственный, нейронный, сеть]
9                                        [россиянин, вступление, вто, пострадать]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [76]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [77]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.12
F1 -  0.11
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [25]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [стандарт, который, источник, фгоса, васильев, ольга, исторический, предмет, результат, образовательный]
1                                        [красота, ваш, отчаяние, глаз, это, уйти, порыв, кошка, значит, один]
2                                 [пепеляев, юзеф, книга, якутия, война, самый, восстание, год, место, леонид]
3                              [команда, гонка, это, пилот, сказать, mclaren, сезон, ferrari, время, mercedes]
4                                          [есенин, поэт, клюев, год, свой, который, жизнь, это, смерть, 1925]
5                      [наш, медицина, кафедра, это, медицинский, выпускник, работать, работа, уровень, рудна]
6                                 [русский, книга, мозг, человек, два, говор, островной, это, анатомия, глава]
7                         [ирак, война, американец, партизанский, это, структура, войско, единый, лишь, новый]
8                      [который, нейросеть, ai, свой, клетка, самый, искусственный, интеллект, основа, задача]
9

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [78]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [79]:
data['content_norm'] = data['content'].apply(normalize)

In [80]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Ещу улучшения!

In [81]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [стандарт, источник, предмет, васильев, результат, фгоса, ольга, школа, письмо, содержание]
1                       [красота, отчаяние, глаз, порыв, кошка, место, руина, предмет, дело, вопрос]
2                      [пепеляев, юзеф, книга, якутия, восстание, война, год, место, леонид, сибирь]
3                       [команда, гонка, пилот, сезон, время, машина, круг, место, гран-при, борьба]
4                      [есенин, поэт, клюев, год, жизнь, смерть, сергей, человек, борода, мариенгоф]
5    [медицина, кафедра, выпускник, работа, уровень, рудна, год, практика, ординатор, преподаватель]
6                 [книга, мозг, человек, говор, анатомия, звезда, глава, вопрос, азербайджан, слово]
7                   [ирак, война, американец, структура, войско, хусейн, оружие, солдат, узел, штаб]
8       [нейросеть, клетка, интеллект, задача, основа, время, изображение, работа, пиксель, человек]
9        [россия, георгиев, кристалина, вступление, вто, банка, страна, директор, слово, пе

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [82]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [83]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [84]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [85]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [86]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [104]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [105]:
keywords[:3]

[['ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'письмо',
  'произведение',
  'редакция',
  'содержание'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'предмет',
  'мечта',
  'безобразие',
  'познание',
  'англичанин'],
 ['якутия',
  'книга',
  'восстание',
  'сибирь',
  'леонид',
  'красных',
  'пустынь',
  'перо',
  'война',
  'мороз']]

In [106]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [90]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [91]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [92]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 34.4 s, sys: 19.3 ms, total: 34.5 s
Wall time: 34.5 s


In [93]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [94]:
keywords_rw.head(10)

0    [произведение, предмет, стандарт, реализация, результат, ольга, письмо, фгоса, источник, содержание]
1                     [красота, глаз, отчаяние, кошка, дело, руина, предмет, разочарование, мечта, порыв]
2                           [пепеляев, книга, юзеф, ночное, мороз, якутия, война, год, сибирь, восстание]
3               [команда, пилот, гонка, дженсон, внимание, время, мальдонадо, сезон, испания, реальность]
4                                [есенин, поэт, клюев, человек, друг, год, сергей, жизнь, смерть, борода]
5            [выпускник, ординатор, специалист, кафедра, работа, практика, медицина, звено, уровень, год]
6                  [книга, азербайджан, анатомия, наука, говор, мозг, человек, язык, голов, исследование]
7                          [ирак, американец, число, война, нападение, лёт, милиция, мосул, орган, взрыв]
8    [нейросеть, клетка, решение, задача, подкрепление, пиксель, фото, время, исследователь, электроника]
9    [георгиев, россия, кристалина, слово, дир

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [95]:
import networkx as nx

In [96]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [97]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 1min 21s, sys: 41 ms, total: 1min 21s
Wall time: 1min 21s


In [98]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [99]:
from gensim.summarization import keywords

In [101]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [103]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


Наша имплементация отработала получше.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).